In [1]:
import numpy as np

from sklearn.neural_network import MLPRegressor

from Chempy.parameter import ModelParameters
from Chempy.cem_function import single_timestep_chempy

import sbi.utils as utils
from sbi.inference.base import infer
from sbi.analysis import pairplot

import torch
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform

import tensorflow as tf

import time as t
import pickle


/Users/bguenes/miniconda3/envs/master_chempy_multi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ----- Set-up priors -----
a = ModelParameters()
priors = torch.tensor([[a.priors[opt][0], a.priors[opt][1]] for opt in a.to_optimize])

combined_priors = utils.MultipleIndependent(
    [Normal(p[0]*torch.ones(1), p[1]*torch.ones(1)) for p in priors] +
    [Uniform(torch.tensor([2.0]), torch.tensor([12.8]))],
    validate_args=False)

In [3]:
def simulator_chempy(sample):
    out = single_timestep_chempy((sample.numpy().tolist(),a))
    return out[0]

In [4]:
# ----- sbi setup -----
num_sim = 10
method = 'SNPE' #SNPE or SNLE or SNRE

start = t.time()
posterior = infer(
    simulator_chempy,
    combined_priors,
    method=method,
    num_simulations=num_sim)

print(f'Time taken to train the posterior with {num_sim} samples: {round(t.time() - start, 4)}s')

Running 10 simulations.: 100%|██████████| 10/10 [00:01<00:00,  8.22it/s]


: 